In [21]:
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate -U

In [22]:
# Imports
import torch
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from string import punctuation
import re
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
# Set up GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [24]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/UofT/IMDB_Dataset.csv' # @Team: Replace with your own google drive path to dataset
df = pd.read_csv(file_path)
df = df.sample(frac=1.0, random_state=413)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,review,sentiment
21315,"First of all, it is interesting to note that o...",positive
20835,The unthinkable has happened. Having first wit...,negative
29274,One of the best records of Israel's response t...,positive
32234,"But, lets face it... it got a few nostalgic si...",negative
26597,Ben a out-of-town cop is convinced his sister ...,negative


In [25]:
# Convert sentiment to binary labels
df.rename(columns={'sentiment': 'labels'}, inplace=True)
label_mapping = {'positive': 1, 'negative': 0}
df['labels'] = df['labels'].map(label_mapping)
df.head()

,review,labels
21315,"First of all, it is interesting to note that o...",1
20835,The unthinkable has happened. Having first wit...,0
29274,One of the best records of Israel's response t...,1
32234,"But, lets face it... it got a few nostalgic si...",0
26597,Ben a out-of-town cop is convinced his sister ...,0


In [26]:
# Function to clean data
def clean_data(text):
    english_stopwords = set(stopwords.words("english"))
    cleaned_text = []
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(f'[{re.escape(punctuation)}]', ' ', text)
    for token in text.split():
        if token.lower() not in english_stopwords and not token.isdigit():
            cleaned_text.append(token.lower())
    return ' '.join(cleaned_text)

# Apply clean_data function to 'review' column
df['review'] = df['review'].apply(lambda x: clean_data(x))

df.head()

,review,labels
21315,first interesting note one users commented fil...,1
20835,unthinkable happened first witnessed years ago...,0
29274,one best records israel response murder rabin ...,1
32234,lets face got nostalgic sighs show consistentl...,0
26597,ben town cop convinced sister brutally killed ...,0


In [27]:
# Import BERT Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=1024)

In [28]:
# Import BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [29]:
# Split dataset into 70% training, 10% validation, 20% testing
train_texts = list(df.review[:35000])
val_texts = list(df.review[35000:40000])
test_texts = list(df.review[40000:])

train_labels = list(df.labels[:35000])
val_labels = list(df.labels[35000:40000])
test_labels = list(df.labels[40000:])

In [30]:
# Tokenize different datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [31]:
# Custom dataset
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [32]:
# Instantiate custom datasets
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [33]:
# Compute accuracy and metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [34]:
# Training arguments with lr = 1e-4
training_args3 = TrainingArguments(
    output_dir='./',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=30,
    logging_steps=50,
    evaluation_strategy="steps",
    learning_rate=1e-4,
    eval_steps=50,
    fp16=True,
    load_best_model_at_end=True
)

# Define trainer with arguments from above
trainer3 = Trainer(
    model=model,
    args=training_args3,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics= compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [35]:
# Fine-tune pretrained Bert Model
trainer3.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.634700,0.373981,0.831000,0.833750,0.831145,0.830693
100,0.491000,0.595497,0.766200,0.825237,0.766880,0.755392
150,0.436800,0.342837,0.853800,0.859159,0.853605,0.853202
200,0.447100,0.431354,0.845200,0.848525,0.845044,0.844787
250,0.455600,0.413748,0.832000,0.832203,0.832039,0.831985
300,0.405000,0.373090,0.858000,0.858092,0.857974,0.857983
350,0.390300,0.376399,0.854800,0.860616,0.855003,0.854261
400,0.464300,0.442300,0.824200,0.848745,0.823775,0.820917
450,0.528200,0.679338,0.498400,0.249200,0.500000,0.332621
500,0.601100,0.425271,0.846000,0.849673,0.845836,0.845550


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory ./checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=5250, training_loss=0.3536335743495396, metrics={'train_runtime': 3575.3818, 'train_samples_per_second': 29.367, 'train_steps_per_second': 1.468, 'total_flos': 2.76266608128e+16, 'train_loss': 0.3536335743495396, 'epoch': 3.0})

In [36]:
# Record test results
results3 = trainer3.predict(test_dataset)
results3

PredictionOutput(predictions=array([[-1.6181641,  1.8271484],
       [-1.3271484,  1.5234375],
       [-1.5976562,  1.8076172],
       ...,
       [-1.2226562,  1.3203125],
       [ 1.1445312, -1.7802734],
       [-1.5087891,  1.7255859]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 1]), metrics={'test_loss': 0.3318052589893341, 'test_accuracy': 0.8768, 'test_precision': 0.8772685254271748, 'test_recall': 0.876705684737767, 'test_f1': 0.8767392521730409, 'test_runtime': 38.7781, 'test_samples_per_second': 257.877, 'test_steps_per_second': 8.613})